In [1]:
from pathlib import Path

import wandb
import torch
from lightning.pytorch import loggers, utilities, Trainer

from model import MNIST
from data_module import MNISTDataModule

c:\Dev\MNIST\.venv\Lib\site-packages\torch\nn\modules\lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [2]:
root_path = Path('../')
dm = MNISTDataModule(data_dir=(root_path / 'data').as_posix())

In [3]:
artifact_dir = root_path / 'artifacts'
artifact_dir.mkdir(exist_ok=True)
model_path = Path(loggers.WandbLogger.download_artifact(
    artifact="sampath017/model-registry/MNIST:v2", 
    artifact_type='model',
    save_dir=artifact_dir
)) / 'model.ckpt' # type: ignore

wandb:   1 of 1 files downloaded.  


In [4]:
model = MNIST.load_from_checkpoint(model_path, map_location=torch.device('cpu'))
utilities.model_summary.ModelSummary(model) # type: ignore

  | Name  | Type       | Params
-------------------------------------
0 | model | Sequential | 71.0 K
-------------------------------------
71.0 K    Trainable params
0         Non-trainable params
71.0 K    Total params
0.284     Total estimated model params size (MB)

In [5]:
trainer = Trainer()

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
c:\Dev\MNIST\.venv\Lib\site-packages\lightning\pytorch\trainer\connectors\logger_connector\logger_connector.py:67: UserWarning: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
  warning_cache.warn(


In [6]:
trainer.test(model, datamodule=dm);

Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃   Runningstage.testing    ┃                           ┃
┃          metric           ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │     99.2699966430664      │
│         test_loss         │   0.025722390040755272    │
└───────────────────────────┴───────────────────────────┘